In [ ]:
import torch
import numpy as np

# # display CUDA devices available
torch.cuda.is_available()


# set device to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

: 

In [ ]:
torch.cuda.is_available()

: 

In [ ]:
# load data
evals = np.load('../../data/evaluations.npy')
turns = np.load('../../data/turns.npy')
one_hot_boards = np.load('../../data/one_hot_boards.npy')


# reshape data to be 2D
evals = evals.reshape(-1, 1)
# normalize data to be / 100
EVAL_DIVISOR = 1000
evals = evals / EVAL_DIVISOR

# shuffle data
np.random.seed(0)
np.random.shuffle(evals)
np.random.seed(0)
np.random.shuffle(turns)
np.random.seed(0)
np.random.shuffle(one_hot_boards)

# split data into train and test
SPLIT_INDEX = int(len(evals) * 0.8)
train_evals = evals[:SPLIT_INDEX]
train_turns = turns[:SPLIT_INDEX]
train_boards = one_hot_boards[:SPLIT_INDEX]

test_evals = evals[SPLIT_INDEX:]
test_turns = turns[SPLIT_INDEX:]
test_boards = one_hot_boards[SPLIT_INDEX:]

# convert to tensors
train_evals = torch.from_numpy(train_evals).float().to(device)
train_turns = torch.from_numpy(train_turns).float().to(device)
train_boards = torch.from_numpy(train_boards).float().to(device)

test_evals = torch.from_numpy(test_evals).float().to(device)
test_turns = torch.from_numpy(test_turns).float().to(device)
test_boards = torch.from_numpy(test_boards).float().to(device)

# # flatten boards and add turns 
# train_boards = train_boards.view(train_boards.shape[0], -1)
# train_boards = torch.cat((train_boards, train_turns), 1)

# test_boards = test_boards.view(test_boards.shape[0], -1)
# test_boards = torch.cat((test_boards, test_turns), 1)

train_boards = train_boards.view(-1, 12,64).transpose(1,2)
test_boards = test_boards.view(-1, 12,64).transpose(1,2)



print(train_boards.shape)

# # print 1 example
# print(train_evals[6])
# print(train_turns[0])
# print(train_boards[0])

: 

In [ ]:
# train model
from model import Transformer

loss = torch.nn.L1Loss()
model = Transformer().to(device)

: 

In [ ]:
# model graphing code
test_loss_data = []
train_loss_data = []


# hyperparameters
EPOCHS = 1000
BATCH_SIZE = 800
LEARNING_RATE = 15e-3

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


# train
for epoch in range(EPOCHS):
  for i in range(0, len(train_evals), BATCH_SIZE):
    # get batch
    batch_evals = train_evals[i:i+BATCH_SIZE].to(device)
    batch_boards = train_boards[i:i+BATCH_SIZE].to(device)
    print(batch_boards.shape)
    # forward pass
    y_pred = model.forward(batch_boards)

    # calculate loss
    loss_val = loss(y_pred, batch_evals)

    # backward pass
    optimizer.zero_grad()
    loss_val.backward()
    optimizer.step()

  # # print loss every 100 epochs
  # if epoch % 50 == 0:
  #   # use test data to calculate loss
  #   y_pred = model.predict(test_boards.to(device))
  #   test_loss = loss(y_pred, test_evals.to(device)).item()*EVAL_DIVISOR

  #   # use train data to calculate loss
  #   y_pred = model.predict(train_boards.to(device))
  #   train_loss = loss(y_pred, train_evals.to(device)).item()*EVAL_DIVISOR
  #   print('Epoch: ', epoch, "test: ", test_loss, "train: ", train_loss)

  #   # save loss data
  #   test_loss_data.append(test_loss)
  #   train_loss_data.append(train_loss)


import matplotlib.pyplot as plt
# plot loss
plt.plot(test_loss_data, label='test')
plt.plot(train_loss_data, label='train')
plt.legend()
plt.show()

: 